In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
from deepface import DeepFace
import pandas as pd
import time
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

### ResNet SSD model based on VGG
https://sefiks.com/2020/08/25/deep-face-detection-with-opencv-in-python/

macOS cannot quit properly in jupyter notebook

In [2]:
deploy_path = "./res_ssd/deploy.prototxt"
model_path = "./res_ssd/res10_300x300_ssd_iter_140000.caffemodel"
deep_face_database = "./database/"
cv_flag = "videoplayback.mp4"
# cv_flag = 0

In [3]:
OFFSET = 20
def detect_and_predict_mask(frame, faceNet):
    # grab the dimensions of the frame and then construct a blob
    # from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 
                                 1.0, 
                                 (224, 224),
                                 (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    locs = []
#     preds = []
    faces = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX-OFFSET), max(0, startY-OFFSET))
            (endX, endY) = (min(w - 1, endX+OFFSET), min(h - 1, endY+OFFSET))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
#             face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
#             face = cv2.resize(face, (64, 64))
            face = cv2.resize(face, (224, 224))
#             face = img_to_array(face)
#             face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    return locs, faces

In [4]:
from deepface.basemodels import VGGFace
vgg_model = VGGFace.loadModel()
def get_deepFace_result(face):
    cv2.imwrite('tmp.jpg', face)
    df = DeepFace.find(img_path="tmp.jpg", 
                       db_path=deep_face_database, 
                       enforce_detection=False, 
                       model_name = 'VGG-Face',
                       model = vgg_model,
                       distance_metric = 'cosine')
#     df = DeepFace.find(img_path="tmp.jpg", 
#                        db_path=deep_face_database, 
#                        enforce_detection=False)
    id = "unknown"
    if df.shape[0] >0:
        id = df.iloc[0].identity
#         print(id)
#         print(os.path.split(id)[0])
        id = os.path.split(id)[0].replace(deep_face_database,'')
    return id

In [ ]:
detector = cv2.dnn.readNetFromCaffe(deploy_path, model_path)
source=cv2.VideoCapture(cv_flag)
while(source.isOpened()):
    ret,image=source.read()
    if ret == False:
        break
    locs, faces = detect_and_predict_mask(image, detector)
    for (box, face) in zip(locs, faces):
        (startX, startY, endX, endY) = box
        color = (0, 255, 0)
        path = get_deepFace_result(face)
        cv2.putText(image, path, (startX, startY - 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.putText(image, "Mask", (startX, startY - 10),
                cv2.FONT_HERSHEY_COMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
        cv2.imshow('frame',image)
    
#     time.sleep(0.2)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()
source.release()

Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

Already built model is passed
There are  3  representations found in  representations_vgg_face.pkl


Analyzing: 100%|██████████| 1/1 [00:00<00:00,  2.58it/s]


find function lasts  0.39354395866394043  seconds


Analyzing: 100%|██████████| 1/1 [00:00<00:00,  5.12it/s]

...

find function lasts  0.1867389678955078  seconds
Already built model is passed
There are  3  representations found in  representations_vgg_face.pkl


Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

find function lasts  0.26938796043395996  seconds
Already built model is passed
There are  3  representations found in  representations_vgg_face.pkl


Analyzing: 100%|██████████| 1/1 [00:00<00:00,  4.74it/s]


find function lasts  0.21511387825012207  seconds
